In [ ]:
#better use this script in a virtual environment or cuda container
!pip install transformers accelerate bitsandbytes peft datasets trl
!pip install flash-attn --no-build-isolation

In [ ]:
import os
import torch

import pandas as pd
import numpy as np

from datasets import load_dataset, load_metric
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, BitsAndBytesConfig, TrainingArguments, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

In [ ]:
path = "../data/processed"
dataset = load_dataset(path)

In [ ]:
MODEL_NAME = "distilbert-base-uncased"

In [ ]:
# quantization config
bitsandbytes_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="fp4",
    bnb_4bit_compute_dtype=torch.float16,  
    llm_int8_enable_fp32_cpu_offload=False,
    llm_int8_has_fp16_weight=True,
)

In [ ]:
df = dataset['train'].to_pandas()
count_of_categories = df['label'].nunique()
print(count_of_categories)

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)

# Ensure tokenizer has an EOS token
if tokenizer.eos_token is None:
    tokenizer.add_special_tokens({'eos_token': '[EOS]'})

# Explicitly add a pad token if it's not set correctly
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

tokenizer.pad_token = tokenizer.eos_token  # Alternatively, you can use the newly added '[PAD]' token
tokenizer.padding_side = "right"

In [ ]:
def tokenize_and_get_lengths(examples):
    tokenized_examples = tokenizer(examples['text'], truncation=False, padding=False)
    return {'length': [len(tokens) for tokens in tokenized_examples['input_ids']]}

measure = dataset.map(tokenize_and_get_lengths, batched=True, remove_columns=['text'])

max_seq_length = max(measure['train']['length'])
print(f'Max lenght of sequence: {max_seq_length}')

In [ ]:
max_seq_length = 512

In [ ]:
train_label = dataset['train']
test_label = dataset['test']

In [ ]:
#need to be refactored, sometimes this method printing errors, sometigmes not...
vocab_size = tokenizer.vocab_size

def preprocess_function(examples):
    tokenized_examples = tokenizer(examples['text'], max_length=max_seq_length,truncation=True, padding=True)
    tokenized_examples['label'] = examples['label']
    # Check if any input_ids exceed the vocabulary size
    for input_ids in tokenized_examples['input_ids']:
        if any(id >= vocab_size for id in input_ids):
            print(f"Input ID out of range: {input_ids}")
    # Keep only input_ids, attention_mask, and label
    processed_examples = {
        'input_ids': tokenized_examples['input_ids'],
        'attention_mask': tokenized_examples['attention_mask'],
        'label': tokenized_examples['label']
    }
    return processed_examples

In [ ]:
tokenized_train = train_label.map(preprocess_function, batched=True,remove_columns=['text'])
tokenized_test = test_label.map(preprocess_function, batched=True, remove_columns=['text'])

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(
    MODEL_NAME,
    quantization_config=bitsandbytes_config,
    attn_implementation="flash_attention_2",
    num_labels=count_of_categories
)

In [ ]:
# little bit a suger for more effeciency
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    target_modules=["attention.q_lin", "attention.k_lin", "attention.v_lin", "attention.out_lin"],  # Correct target modules
    task_type="CAUSAL_LM"
)

In [ ]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
args = TrainingArguments(
  output_dir = MODEL_NAME,
  num_train_epochs=5,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,  warmup_steps = 0.03,
  logging_steps=10,
  weight_decay=0.01,
  save_strategy="epoch",
  eval_strategy="epoch",
  learning_rate=2.5e-5,
  bf16=False,
)

In [ ]:
max_seq_length = 512

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  dataset_text_field="text",
  tokenizer=tokenizer,
  packing=True,
  args=args,
  train_dataset=tokenized_train,
  eval_dataset=tokenized_test,
  data_collator=data_collator,
  compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()